In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [2]:
from ta.trend import SMAIndicator
def SMA(window_para,to_csv):
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    result.loc[result['code'] == result['code'].shift(window_para),'sma'] = SMAIndicator(close=result['close_weekly'],window=window_para).sma_indicator()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']>result['sma']),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['close_weekly']<result['sma']),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checksma.csv')
    cul_arr = day_return['cumulative'].to_list()
    dd_arr = []
    maxcul = 0
    for x in day_return.cumulative.dropna().tolist():
        if x >= maxcul:
            maxcul = x
            dd_arr.append(0)
        else:
            dd_arr.append((maxcul-x)/maxcul)
    print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para

In [5]:
SMA(45,to_csv = False)

MDD 0.3495627015105781


(1.9423422946803095, 45)

In [3]:
optimize_result = []
for window_para in range(2,50, 1):
    x=SMA(window_para,to_csv = False)
    optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

MDD 0.41926731748491874
MDD 0.44960705387302885
MDD 0.44578365406771603
MDD 0.45392574324802326
MDD 0.4386334100949946
MDD 0.4175249765671521
MDD 0.41640597824698666
MDD 0.41857769714671106
MDD 0.42753738271374486
MDD 0.415742806273866
MDD 0.4105525060983592
MDD 0.39880685044319336
MDD 0.38829327796303836
MDD 0.3907980826742448
MDD 0.40044252272581826
MDD 0.4069003139521242
MDD 0.4055810463358774
MDD 0.406473992585331
MDD 0.40571264331617407
MDD 0.40545122594563915
MDD 0.4099666666395092
MDD 0.40477473770723127
MDD 0.4051780127176718
MDD 0.40445207350392864
MDD 0.3954804879648296
MDD 0.394952758805046
MDD 0.390011591253684
MDD 0.3962262332333353
MDD 0.394028544903807
MDD 0.3913966336021713
MDD 0.3935057513678032
MDD 0.38788697023613017
MDD 0.38033223551686707
MDD 0.3825297702442774
MDD 0.37898574498329207
MDD 0.37499261505085724
MDD 0.3674701640990456
MDD 0.36238967026518787
MDD 0.35932082206275806
MDD 0.3526464342922621
MDD 0.3565777377934631
MDD 0.3525201802752186
MDD 0.3516155966259

In [4]:
optimize_result

,0,1
0,0.859610,2
1,0.937867,3
2,1.001317,4
3,1.055352,5
4,1.094934,6
5,1.199412,7
6,1.306938,8
7,1.367965,9
8,1.320400,10
9,1.393191,11
